In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

In [ ]:
data = pd.read_csv("eleicoes_2006_a_2010.csv")

matplotlib.rcParams['figure.figsize'] = (14.0, 8.0)

In [ ]:
data.head()

In [ ]:
df_votos = pd.DataFrame({"votos":data["votos"], "log(votos + 1)":np.log1p(data["votos"])})
df_votos.hist(log=True)

df_qtdDoacoes = pd.DataFrame({"qtd. Doacoes":data["quantidade_doacoes"], "log(qtd. Doacoes + 1)":np.log1p(data["quantidade_doacoes"])})
df_qtdDoacoes.hist(log=True)

df_qtdDoadores = pd.DataFrame({"qtd. Doadores":data["quantidade_doadores"], "log(qtd. Doadores + 1)":np.log1p(data["quantidade_doadores"])})
df_qtdDoadores.hist(log=True)

df_totalReceita = pd.DataFrame({"total receita":data["total_receita"], "log(total receita + 1)":np.log1p(data["total_receita"])})
df_totalReceita.hist(log=True)

df_recursosOutros = pd.DataFrame({"recursos de outros":data["recursos_de_outros_candidatos/comites"], "log(recursos de outros + 1)":np.log1p(data["recursos_de_outros_candidatos/comites"])})
df_recursosOutros.hist(log=True)

df_qtdDespesas = pd.DataFrame({"qtd. Despesas":data["quantidade_despesas"], "log(qtd. Despesas + 1)":np.log1p(data["quantidade_despesas"])})
df_qtdDespesas.hist(log=True)

df_qtdFornecedores = pd.DataFrame({"qtd. Fornecedores":data["quantidade_fornecedores"], "log(qtd. Fornecedores + 1)":np.log1p(data["quantidade_fornecedores"])})
df_qtdFornecedores.hist(log=True)

df_totalDespesas = pd.DataFrame({"total despesa":data["total_despesa"], "log(total despesa + 1)":np.log1p(data["total_despesa"])})
df_totalDespesas.hist(log=True)

In [ ]:
data = data.drop('media_receita', axis=1)
data = data.drop('media_despesa', axis=1)
data.head()

In [ ]:
#log transform skewed numeric features
numeric_feats = data.dtypes[data.dtypes != "object"].index

numeric_feats = numeric_feats.drop('ano') #não aplico normalização na variável ano pois ela é categórica

data[numeric_feats] = np.log1p(data[numeric_feats])

In [ ]:
from sklearn.model_selection import train_test_split
entradas = data.loc[:,:'ocupacao']
saida = data.loc[:,'votos']

entradas = entradas.fillna(entradas.mean())

X_train, X_test, Y_train, Y_test = train_test_split(entradas,saida)

In [ ]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, Y_train, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
reg_linear = LinearRegression()
reg_linear.fit(X_train, Y_train)
mean_squared_error(Y_train, reg_linear.predict(X_train))

In [ ]:
from sklearn.linear_model import Ridge
alphas_ridge = np.arange(0.5,12,0.5).tolist()
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas_ridge]
cv_ridge = pd.Series(cv_ridge, index = alphas_ridge)
cv_ridge.plot(title = "Validation - Just Do It")
plt.xlabel("alpha")
plt.ylabel("rmse")